In [1]:
import pandas as pd
import numpy as np
import dash
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc

In [2]:
#external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
external_stylesheets = [dbc.themes.SUPERHERO]

## Initialize the app

In [3]:
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)
#app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

## Specify the contents of the dashboard

In [4]:
app.layout = html.Div(
[
    html.H1("Bill and Ted's Excellent Dashboard"),
    
    dcc.Input(id = 'username', type = 'text', placeholder = 'Please type your name here'),
    
    dcc.Markdown(id = 'countletters')
]

)

## Callbacks

In [5]:
@app.callback(Output(component_id = 'countletters', component_property = 'children'), 
              Input(component_id = 'username', component_property = 'value'))

def countletters(name):
    n = str(name).replace(' ','')
    return f"You have {len(n)} letters in your name dude!"




## Run the app

In [6]:
if __name__== "__main__":
    app.run_server(mode= 'external', host = "0.0.0.0", debug=True)

Dash app running on http://0.0.0.0:8050/
